In [3]:
import pandas as pd
#data = pd.read_csv('Twitter_Data.csv')
#data1 = data.groupby('category').sample(n=10000, replace=False)
data = pd.read_csv('train.csv')

In [4]:
data['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [3]:
data.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
y = data['target']
X = data['text']

In [5]:
print(y.shape)
print(X.shape)
print(type(X[1]))

(7613,)
(7613,)
<class 'str'>


def mapNanValues(data):
    nan_values = []
    for i in range(len(data)):
        if type(data[i]) == float:
            nan_values.append(i)
    return nan_values

In [6]:
import numpy as np
import tensorflow as tf

In [7]:
# values = mapNanValues(X)
# print(values)
# y = y.drop(index=values).reset_index(drop=True)
# X = X.drop(index=values).reset_index(drop=True)
# print(y.shape)
# print(X.shape)
# zeros = np.zeros((len(y), 3))
# ##### One Hot labels ####
# for i in range(len(y)):
#     if y[i] == -1:
#         zeros[i][0] = 1
#     elif y[i] == 0:
#         zeros[i][1] = 1
#     elif y[i] == 1:
#         zeros[i][2] = 1
# y = zeros

In [8]:
#data1.groupby('category').count()

In [7]:
from tensorflow.keras.layers import LSTM, Dense, Softmax, Dropout, Input, Embedding, TextVectorization
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall, BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras import Model
#from tensorflow.keras.preprocessing.text import one_hot
#from tensorflow.keras.preprocessing.sequence import pad_sequences
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=False)

In [14]:
def Bert_LSTM():
    text_input = Input(shape=(), dtype=tf.string, name='text')
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)
    l = LSTM(64, input_shape=(), activation='relu', dropout=0.1)(outputs['sequence_output'])
    output = Dense(1, activation='sigmoid')(l)
    model = Model(inputs=[text_input], outputs = [output])
    return model

In [8]:
# def Trainable_Embedding_LSTM(data):
#     intencoding = TextVectorization(max_tokens=999, output_mode='int', output_sequence_length=40)
#     intencoding.adapt(data)
#     intencoding.compile()
#     text_input = Input(shape=(), dtype=tf.string, name='text')
#     textvec = intencoding(text_input)
#     l = Embedding(input_dim=999, output_dim=100)(textvec)
#     l = LSTM(128, input_shape=(40, 100), activation='relu', return_sequences=True)(l)
#     l = Dropout(0.1)(l)
#     l = LSTM(128, activation='relu', return_sequences=False)(l)
#     l = Dropout(0.1)(l)
#     l = Dense(32)(l)
#     output = Dense(3, activation='softmax')(l)
#     model = Model(inputs=[text_input], outputs = [output])
#     return model


In [ ]:
#from sklearn import model_selection
#X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=35)

In [15]:
METRICS = [
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall')  
]
optimizer=Adam(learning_rate=3e-4)
model2 = Bert_LSTM()
model2.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=METRICS)
history_bert = model2.fit(X, y, batch_size = 64, epochs=50, verbose=1, validation_split=0.15, callbacks=[EarlyStopping(monitor='val_loss', patience=3)])
model2.save('bert_model.model')

Epoch 1/50
102/102 [==============================] - 1876s 18s/step - loss: 0.8628 - accuracy: 0.7200 - precision: 0.6830 - recall: 0.6316 - val_loss: 0.5482 - val_accuracy: 0.7539 - val_precision: 0.7642 - val_recall: 0.6823
Epoch 2/50
 94/102 [==========================>...] - ETA: 2:02 - loss: 0.5570 - accuracy: 0.7342 - precision: 0.7225 - recall: 0.6067

KeyboardInterrupt: 

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history_bert.history['accuracy'])
plt.plot(history_bert.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Bert_Plot.png', format='png')

In [12]:
METRICS = [
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall')
]
optimizer=Adam(learning_rate=3e-4)
model3 = Trainable_Embedding_LSTM(X)
model3.compile(optimizer=optimizer, loss='categorical_crossentropy', validation_split=0.2, metrics=METRICS)
history_custom = model3.fit(X, y, batch_size = 64, epochs=50, verbose=1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])
model3.save('custom_model.model')

TypeError: Invalid keyword argument(s) in `compile()`: ({'validation_split'},). Valid keyword arguments include "cloning", "experimental_run_tf_function", "distribute", "target_tensors", or "sample_weight_mode".

In [ ]:
# plt.plot(history_custom.history['accuracy'])
# plt.plot(history_custom.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# plt.savefig('Custom_Plot.png', format='png')